<a href="https://colab.research.google.com/github/msfasha/307401-Big-Data/blob/main/20251/Module_6_introduction_to_databricks/1_NYC_Taxi_Analytics_Statistics_and_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analytics on the NYC Taxi Dataset Using Parquet & Spark

This notebook provides ready-to-use analytical, statistical, and machine-learning workflows on the NYC Taxi dataset stored in Parquet format using Apache Spark.

## 1. Dataset Motivation

The NYC Taxi dataset combines time-based, numerical, and categorical data at scale, making it suitable for statistical analysis and machine learning.

## 2. Load Data from Parquet

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os

spark = SparkSession.builder.appName("NYTaxiAnalytics").getOrCreate()

# Define the URL and local path for the dataset
dataset_url = "https://github.com/msfasha/307401-Big-Data/raw/main/datasets/yellow_tripdata_2025-10.parquet"
local_parquet_path = "yellow_tripdata_2025-10.parquet"

# Download the file if it doesn't exist locally
if not os.path.exists(local_parquet_path):
    print(f"Downloading {dataset_url} to {local_parquet_path}...")
    # Using subprocess.run for better control and error handling than !wget
    import subprocess
    try:
        subprocess.run(["wget", dataset_url], check=True)
        print("Download complete.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading file: {e}")
        # Handle the error, maybe raise it again or exit
        raise

# Read the Parquet file from the local path
taxi_df = spark.read.parquet(local_parquet_path)

## 3. Schema Inspection and Profiling

In [7]:
taxi_df.printSchema()
taxi_df.count()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



4428699

## 4. Summary Statistics

In [8]:
taxi_df.select(
    "trip_distance",
    "fare_amount",
    "passenger_count"
).describe().show()

+-------+------------------+------------------+------------------+
|summary|     trip_distance|       fare_amount|   passenger_count|
+-------+------------------+------------------+------------------+
|  count|           4428699|           4428699|           3437812|
|   mean|6.6967439196033824|18.255545755085404|1.2748128751659487|
| stddev| 656.9619547390072|19.688461694744408|0.6913772296675526|
|    min|               0.0|            -800.0|                 0|
|    max|         276333.48|            1071.9|                 9|
+-------+------------------+------------------+------------------+



## 5. Distribution Analysis

In [9]:
from pyspark.sql.functions import floor

taxi_df.withColumn(
    "fare_bucket", floor(col("fare_amount") / 5) * 5
).groupBy("fare_bucket").count().orderBy("fare_bucket").show()

+-----------+-----+
|fare_bucket|count|
+-----------+-----+
|       -800|    3|
|       -795|    1|
|       -750|    1|
|       -700|    2|
|       -665|    1|
|       -650|    1|
|       -645|    1|
|       -640|    1|
|       -620|    1|
|       -600|    3|
|       -550|    1|
|       -520|    1|
|       -500|   14|
|       -495|    2|
|       -490|    1|
|       -480|    4|
|       -475|    1|
|       -455|    1|
|       -450|    2|
|       -420|    2|
+-----------+-----+
only showing top 20 rows


## 6. Time-Based Analysis

In [11]:
from pyspark.sql.functions import hour

taxi_df.withColumn(
    "pickup_hour", hour("tpep_pickup_datetime")
).groupBy("pickup_hour").count().orderBy("pickup_hour").show()

+-----------+------+
|pickup_hour| count|
+-----------+------+
|          0|131173|
|          1| 85061|
|          2| 56379|
|          3| 37668|
|          4| 30050|
|          5| 34947|
|          6| 72168|
|          7|135906|
|          8|179024|
|          9|189819|
|         10|194409|
|         11|205732|
|         12|218286|
|         13|227499|
|         14|244760|
|         15|253651|
|         16|261671|
|         17|295831|
|         18|315195|
|         19|287385|
+-----------+------+
only showing top 20 rows


## 7. Feature Engineering

In [22]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col

taxi_df = taxi_df.withColumn(
    "trip_duration_min",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
)

clean_df = taxi_df.filter(
    (col("trip_duration_min").isNotNull()) &
    (col("trip_duration_min") > 1) &
    (col("trip_duration_min") < 180) &
    (col("fare_amount").isNotNull()) &
    (col("fare_amount") > 0) &
    (col("trip_distance").isNotNull()) &
    (col("trip_distance") > 0) &
    (col("passenger_count").isNotNull()) # Added filter for null passenger_count
)

clean_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee| trip_duration_min|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|       1| 2025-10-01 00:15:32|  2025-10-01 01:04:03|              1|    

## 8. Regression: Fare Prediction

In [18]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["trip_distance", "trip_duration_min", "passenger_count"],
    outputCol="features"
)

ml_df = assembler.transform(clean_df).select("features", "fare_amount")
train_df, test_df = ml_df.randomSplit([0.8, 0.2], seed=42)

In [19]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="fare_amount")
lr_model = lr.fit(train_df)
predictions = lr_model.transform(test_df)

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator

RegressionEvaluator(
    labelCol="fare_amount",
    predictionCol="prediction",
    metricName="rmse"
).evaluate(predictions)

7.497843749737397

## 9. Classification: High Fare Trips

In [24]:
from pyspark.sql.functions import when

labeled_df = clean_df.withColumn(
    "high_fare", when(col("fare_amount") > 50, 1).otherwise(0)
)

final_df = assembler.transform(labeled_df).select("features", "high_fare")
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

In [25]:
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(labelCol="high_fare")
model = log_reg.fit(train_df)
predictions = model.transform(test_df)

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

BinaryClassificationEvaluator(
    labelCol="high_fare",
    metricName="areaUnderROC"
).evaluate(predictions)

0.9855387915472841

## 10. Clustering Trips

In [27]:
from pyspark.ml.clustering import KMeans

cluster_df = assembler.transform(clean_df).select("features")

kmeans = KMeans(k=5, seed=42)
model = kmeans.fit(cluster_df)

model.clusterCenters()

[array([1.22969392, 8.36819633, 1.25657773]),
 array([17.05177296, 89.3303419 ,  1.29901011]),
 array([ 3.11243249, 19.39885216,  1.27419064]),
 array([ 7.81761231, 34.23739602,  1.29047851]),
 array([14.57150232, 56.12243539,  1.3372902 ])]

## Summary

This notebook demonstrated scalable statistical analysis and machine-learning workflows using Spark and Parquet on the NYC Taxi dataset.